<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/CNN_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd 
import sklearn
import gc
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import metrics

Mounted at /content/drive


# Prepare Data

In [2]:
def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    # X = np.array(X)
    return X

def scale_down(X):
  if(X.shape[0]<70000):
    return X
  X_new = []
  for i in range(int(X.shape[0]/2)):
    cur_row = X[i*2]
    for j in range(1,2):
      if(i+j < X.shape[0]):
        cur_row += X[i+j]
    cur_row = cur_row/2.0
    X_new.append(cur_row)
  X_new = np.array(X_new)
  return X_new

def decrease_size(X):
  size = 30000
  if(X.shape[0]<size):
    dif = size - X.shape[0] 
    temp = np.zeros((dif,X.shape[1]))
    X = np.concatenate((X,temp),axis = 0)
  if(X.shape[0]>size):
    X = X[:30000]
  return X

def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train<10]
  X_train_1 = X_train[Y_train>=10]

  Y_train_1 = Y_train[Y_train>=10]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  #X_train = np.array(X_train)
  #Y_train = np.array(Y_train)

  X_train = np.asarray(X_train).astype(np.float32)
  Y_train = np.asarray(Y_train).astype(np.float32)

  return X_train,Y_train


dev = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/devdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
test = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/testdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
train = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/traindata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]

X_train = []
Y_train = []

X_test = []
Y_test = []
size = 40000

for i in range(len(test)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/300_P/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_test.append(X_temp)
    Y_test.append(test[i][2])

X_test = np.array(X_test)
Y_test = np.array(Y_test)      
test = []
gc.collect()

for i in range(len(train)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/303_P/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_train.append(X_temp)
    Y_train.append(train[i][2])

train = []
gc.collect()

for i in range(len(dev)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/302_P/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_train.append(X_temp)
    Y_train.append(dev[i][2])

X_temp = []
dev = []
gc.collect()

X_train = np.array(X_train, dtype = object)
Y_train = np.array(Y_train)

X_upsample, Y_upsample = upsample(X_train,Y_train)
# print(X_upsample.shape)
X_train = []
Y_train = []
gc.collect()

0

# CNN Model For Audio

In [3]:
class CNN_audio:

  def __init__(self):
# Initialising the CNN
    classifier = Sequential()
# Step 1 - Convolution
    classifier.add(Conv1D(60, 10, input_shape = (40000, 74), activation = 'relu'))
# Step 2 - Pooling
    classifier.add(MaxPooling1D(pool_size = 3))
# Adding a second convolutional layer
    classifier.add(Conv1D(30, 5, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
    classifier.add(Conv1D(15, 5, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
# Step 3 - Flattening
    classifier.add(Flatten())
    classifier.add(Dropout(0.5))
# Step 4 - Full connection
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
# classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
    lf = keras.losses.MeanSquaredLogarithmicError()
    classifier.compile(optimizer = 'adam', loss = lf, metrics = [keras.metrics.MeanSquaredError(), keras.metrics.AUC()])
    self.classifier = classifier
  
  def modelFit(self, X, Y, epoch = 10):
    self.classifier.fit(X, Y, epochs=epoch)

  def modelPredict(self, X):
    return self.classifier.predict(X)
  
  def modelEvaluate(self,X,Y):
    return self.classifier.evaluate(X,Y,verbose=1)

model = CNN_audio()
model.modelFit(X_upsample, Y_upsample, 30)
Y_pred = model.modelPredict(X_test)
# Y_pred = Thresholding(model.modelPredict(X_test), 0.8)
# print(classification_report(Y_test,Y_pred))

Epoch 1/30
3/3 [==============================] - 16s 617ms/step - loss: 2.4054 - mean_squared_error: 89.2450 - auc: 0.5000
Epoch 2/30
3/3 [==============================] - 1s 231ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 3/30
3/3 [==============================] - 1s 285ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 4/30
3/3 [==============================] - 1s 275ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 5/30
3/3 [==============================] - 1s 205ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 6/30
3/3 [==============================] - 1s 210ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 7/30
3/3 [==============================] - 1s 210ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 8/30
3/3 [==============================] - 1s 207ms/step - loss: 2.4053 - mean_squared_error: 89.2436 - auc: 0.5000
Epoch 9/30
3/3 

In [4]:
scores = model.modelEvaluate(X_test, Y_test)
print("Loss:", scores)

1/1 [==============================] - 1s 705ms/step - loss: 2.2363 - mean_squared_error: 102.9200 - auc: 0.5000
Loss: [2.236299514770508, 102.91999816894531, 0.5]


In [ ]:
# classifier = Sequential()

# # Step 1 - Convolution
# classifier.add(Conv1D(32, 3, input_shape = (6000, 74), activation = 'relu'))

# # Step 2 - Pooling
# # classifier.add(MaxPooling1D(pool_size = 2))

# # Adding a second convolutional layer
# classifier.add(Conv1D(32, 3, activation = 'relu'))
# classifier.add(MaxPooling1D(pool_size = 2))

# # Step 3 - Flattening
# classifier.add(Flatten())

# # Step 4 - Full connection
# classifier.add(Dense(units = 128, activation = 'relu'))
# classifier.add(Dense(units = 1, activation = 'sigmoid'))

# # Compiling the CNN
# lf = keras.losses.MeanSquaredLogarithmicError()
# classifier.compile(optimizer = 'adam', loss = lf, metrics = [keras.metrics.MeanSquaredError(), keras.metrics.AUC()])

# classifier.fit(X_train, Y_train, epochs=30)
# score = classifier.evaluate(X_test, Y_test)